# Exercises

<div class="alert alert-block alert-info">

**1. Euler-Cromer**

A simple change to the first-order Euler method is to update the position first and then use the new position to evalue the acceleration before updating the velocity.  In this sense, we are treating the velocity update implicitly.
    
</div>

<div class="alert alert-block alert-info">

**2. Chaos**

The equations of motion for a double pendulum consisting of two equal length rigid rods are:
    
\begin{eqnarray}
    \dot{\theta}_1 &=& \frac{6}{I} \frac{2 {p_\theta}_1 - 3 \cos(\theta_1 - \theta_2) {p_\theta}_2}{16 - 9 \cos^2 (\theta_1 - \theta_2)} \\
    \dot{\theta}_2 &=& \frac{6}{I} \frac{2 {p_\theta}_2 - 3 \cos(\theta_1 - \theta_2) {p_\theta}_1}{16 - 9 \cos^2 (\theta_1 - \theta_2)} \\
    {\dot{p}_\theta}_1 &=& -\frac{1}{2} I \left (\dot{\theta}_1\dot{\theta}_2 \sin(\theta_1 - \theta_2) + 3 \frac{g}{l} \sin \theta_1 \right ) \\
    {\dot{p}_\theta}_2 &=& -\frac{1}{2} I \left (-\dot{\theta}_1\dot{\theta}_2 \sin(\theta_1 - \theta_2) +  \frac{g}{l} \sin \theta_2 \right )
\end{eqnarray}

where $I = m l^2$ is the moment of inertia, $\theta_1$ and $\theta_2$ are
the angles of the two rods with respect to the vertical and ${p_\theta}_1$
    and ${p_\theta}_2$ are generalized momenta.  See the [wikipedia page on double pendulum](https://en.wikipedia.org/wiki/Double_pendulum) for a derivation.

Write a routine to integrate this system using RK4, given initial angles
$\theta_1$ and $\theta_2$ (take the initial momenta to be 0).
    
    
A double pendulum system is nonlinear and can exhibit chaotic behavior&mdash;the solution at some time in the future depends sensitively on the initial conditions.  Make a plot of the solution you get ($\theta_1$ and $\theta_2$ vs. time) with two very closely spaced sets of initial conditions.  What happens to the evolution?
    

    
    
</div>